# 07wk-2: Pandas – `pivot_table`, `groupby`+`agg`

최규빈  
2023-10-16

<a href="https://colab.research.google.com/github/guebin/DV2023/blob/main/posts/07wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-wtcnyfm_V0vIvzVnnj8W0J&si=ndZWtP-XX79IklkD>

# 2. Imports

In [138]:
import pandas as pd 
import numpy as np

# 3. pivot_table, groupby+agg

## A. intro

`-` 개념: 그룹화 $\to$ 집계

`# 예제1`: 아래의 예제에서 (학과,성별)로 count의 합계를 구하라.

In [139]:
df=pd.read_csv("https://raw.githubusercontent.com/guebin/DV2022/master/posts/Simpson.csv",index_col=0,header=[0,1])\
.stack().stack().reset_index()\
.rename({'level_0':'department','level_1':'result','level_2':'gender',0:'count'},axis=1)
df

In [140]:
df.pivot_table(index=['department','gender'],values='count',aggfunc='sum')

`#`

`-` 예시에서 본 작업은 아래의 작업들로 세분화 할 수 있다.

1.  그룹화(쿼리): 하나의 dataframe을 sub-dataframe으로 나누는 과정 –
    전체자료를 (학과,성별)로 묶어 총 10개의 sub-dataframe을 만듦
2.  각각집계(각각계산): 나누어진 sub-dataframe에서 어떠한 계산을 각각
    수행함. – 나누어진 sub-dataframe에서 지원자수의 합계를 각각 구함

`-` 위의 같은 작업을 하려면 아래와 같은 요소들이 필요하다.

1.  그룹변수[1] – 그룹화를 위해 필요한 변수, dataframe을
    sub-dataframe으로 나누는 역할.
2.  집계변수[2] – 집계함수의 대상이 되는 변수.
3.  집계변수 – 그룹화된 데이터프레임에 수행하는 계산을 정의하는 함수.

## B. `pivot_table`의 문법

`-` pivot_table의 문법

``` python
df.pivot_table(
    index = 그룹변수
    colums = 그룹변수 
    values = 집계변수
    aggfunc = 집계함수
)
```

`-` 그룹변수: string, 혹은 list of string 으로 전달한다.

-   예시: ‘department’, \[‘department’\], \[‘department’,‘gender’\]

`-` 집계변수: string, 혹은 list of string 으로 전달한다.

-   예시: ‘CANCELLED’, \[‘CANCELLED’\], \[‘CANCELLED’,‘AIR_TIME’\]

`-` 집계함수: 함수자체[3]를 전달하거나, 함수를 의미하는 문자열[4], 혹은
그것들의 리스트형태로 전달한다.

`# 예시`: 집계합수를 전달하는 방법

**data**

[1] 이건 없는 용어에요

[2] 이것도 없는 용어에요

[3] `np.mean`,`sum`

[4] ‘count’, ‘sum’, ‘mean’, ‘median’, ‘min’, ‘max’, ‘std’, ‘var’

In [141]:
df = pd.DataFrame({'category':['A']*5+['B']*5, 'value':np.concatenate([np.random.randn(5), np.random.randn(5)+10])})
df

**방법1** – 함수자체를 전달

In [142]:
df.pivot_table(index=['category'],values='value',aggfunc=np.sum) # 함수자체

/tmp/ipykernel_3437616/3923531937.py:1: FutureWarning: The provided callable <function sum at 0x7f2bb810b1c0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.pivot_table(index=['category'],values='value',aggfunc=np.sum) # 함수자체

**방법2** – 함수를 의미하는 문자열을 전달

In [143]:
df.pivot_table(index=['category'],values='value',aggfunc='sum')  # 리스트

**방법3** – 리스트를 전달

In [144]:
df.pivot_table(
    index=['category'],
    values='value',
    aggfunc=['sum','min',np.mean,np.max,'count']
)

/tmp/ipykernel_3437616/1880687379.py:1: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(
/tmp/ipykernel_3437616/1880687379.py:1: FutureWarning: The provided callable <function amax at 0x7f2bb810ba30> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df.pivot_table(

## C. `groupby` + `aggregate` 의 문법

`-` groupby + aggregate 의 문법

``` python
df.groupby(그룹변수).aggregate({집계변수:집계함수})
```

# 4. AIRLINE 자료로 연습

In [145]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB

`-` 각 변수들에 대한 설명은 아래와 같다. (ChatGPT의 도움을 받아 정리함)

1.  `MONTH`: 비행이 이루어진 월을 나타냄. 1에서 12 사이의 값을 갖음.
2.  `DAY`: 비행이 이루어진 일자를 나타냄. 월에 따라 1~28/29/30/31 사이의
    값을 가질 수 있음.
3.  `WEEKDAY`: 비행이 이루어진 요일을 나타냄. 일반적으로 1(일요일)부터
    7(토요일)까지의 값을 갖음.
4.  `AIRLINE`: 해당 항공편을 운영하는 항공사의 약어나 코드를 나타냄.
5.  `ORG_AIR`: 비행기가 출발하는 공항의 약어나 코드를 나타냄.
6.  `DEST_AIR`: 비행기가 도착하는 공항의 약어나 코드를 나타냄.
7.  `SCHED_DEP`: 원래의 예정된 출발 시간을 나타냄. 시간은 일반적으로
    HHMM 형식으로 표시될 수 있음.
8.  `DEP_DELAY`: 출발 지연 시간을 나타냄. 음수 값은 조기 출발, 양수 값은
    지연을 의미함.
9.  `AIR_TIME`: 실제 공중에서 비행한 시간을 분 단위로 나타냄.
10. `DIST`: 비행 거리를 나타냄. 일반적으로 마일 또는 킬로미터로 표시됨.
11. `SCHED_ARR`: 원래의 예정된 도착 시간을 나타냄. `SCHED_DEP`와 같은
    형식으로 표시될 수 있음.
12. `ARR_DELAY`: 도착 지연 시간을 나타냄. 음수는 조기 도착, 양수는
    지연을 의미함.
13. `DIVERTED`: 항공편이 다른 곳으로 우회되었는지를 나타냄. 1은 우회,
    0은 정상 경로를 의미함.
14. `CANCELLED`: 항공편이 취소되었는지 여부를 나타냄. 1은 취소, 0은
    취소되지 않음을 의미함.

`# 예제1`: 항공사별로 도착지연시간의 평균을 구하라.

`-` 풀이1

In [146]:
df.pivot_table(index='AIRLINE',values='ARR_DELAY')

`-` 풀이2

In [147]:
df.groupby(by="AIRLINE").aggregate({'ARR_DELAY':np.mean})

/tmp/ipykernel_3437616/4188396604.py:1: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby(by="AIRLINE").aggregate({'ARR_DELAY':np.mean})

`#`

`# 예제2`: 항공사별로 비행취소건수의 합계를 구하라. 취소건수가 높은
항공사순으로 정렬하라.

`-` 풀이1: `.pivot_table()`을 이용

In [148]:
df.pivot_table(index='AIRLINE',values='CANCELLED',aggfunc='sum').sort_values('CANCELLED',ascending=False)

`-` 풀이2: `.groupby()`+`.aggregate()`를 이용

In [149]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'sum'}).sort_values('CANCELLED',ascending=False)

`# 예제3`: 항공사별로 비행취소율을 구하라. 비행취소율이 가장 높은 항공사
순으로 정렬하라.

`-` 풀이1: `.pivot_table()`을 이용

In [150]:
df.pivot_table(index='AIRLINE',values='CANCELLED',aggfunc='mean').sort_values('CANCELLED',ascending=False)

`-` 풀이2: `.groupby()`+`.aggregate()`를 이용

In [151]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'mean'}).sort_values('CANCELLED',ascending=False)

`# 예제4` (항공사,요일)별 비행취소건수와 비행취소율을 조사하라.

`-` 풀이1: `.pivot_table()`을 이용

In [152]:
df.pivot_table(index=['AIRLINE','WEEKDAY'],values='CANCELLED',aggfunc=[np.mean,sum])

/tmp/ipykernel_3437616/757677508.py:1: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(index=['AIRLINE','WEEKDAY'],values='CANCELLED',aggfunc=[np.mean,sum])
/tmp/ipykernel_3437616/757677508.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.pivot_table(index=['AIRLINE','WEEKDAY'],values='CANCELLED',aggfunc=[np.mean,sum])

`-` 풀이2: `.groupby()`+`.aggregate()`를 이용

In [153]:
df.groupby(['AIRLINE','WEEKDAY']).aggregate({'CANCELLED':[np.mean,sum]})

/tmp/ipykernel_3437616/1354178761.py:1: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby(['AIRLINE','WEEKDAY']).aggregate({'CANCELLED':[np.mean,sum]})
/tmp/ipykernel_3437616/1354178761.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.groupby(['AIRLINE','WEEKDAY']).aggregate({'CANCELLED':[np.mean,sum]})

`# 예제4`: (항공사,요일)별로 `CANCELLED`는 평균과 합계를 구하고 (즉
비행취소건수와 취소율을 구하고), `AIR_TIME`은 평균과 표준편차를
구하여라.

`-` 풀이1: `.pivot_table()` –\> 이거 제가 수업할때는 못한다고 했는데
찾아보니까 아래처럼 할 수 있습니다.

In [160]:
df.pivot_table(
    index=['AIRLINE', 'WEEKDAY'],
    aggfunc={'CANCELLED': [np.mean, sum], 'AIR_TIME': ['mean','std']}
)

/tmp/ipykernel_3437616/2860571331.py:1: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(
/tmp/ipykernel_3437616/2860571331.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.pivot_table(

`-` 풀이2: `.groupby()`+`.aggregate()`를 이용

In [161]:
df.groupby(['AIRLINE','WEEKDAY'])\
.aggregate({'CANCELLED': [np.mean,sum],'AIR_TIME': ['mean','std']})

/tmp/ipykernel_3437616/2787668343.py:2: FutureWarning: The provided callable <function mean at 0x7f2bb8110670> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate({'CANCELLED': [np.mean,sum],'AIR_TIME': ['mean','std']})
/tmp/ipykernel_3437616/2787668343.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .aggregate({'CANCELLED': [np.mean,sum],'AIR_TIME': ['mean','std']})

`#`

`# 예제5`: 운행구간을 그룹화하고, 운행구간별 비행취소건수와 취소율을
구하여라.

`-` 풀이1

In [156]:
df.assign(DIST_CUT= pd.qcut(df.DIST,q=4)).pivot_table(
    index= ['DIST_CUT'],
    values= 'CANCELLED',
    aggfunc= ['mean','sum']
)

`-` 풀이2

In [157]:
df.assign(DIST_CUT= pd.qcut(df.DIST,q=4))\
.groupby('DIST_CUT')\
.aggregate({'CANCELLED':['mean','sum']})

/tmp/ipykernel_3437616/402182245.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('DIST_CUT')\

`#`

# 5. HW

In [172]:
np.random.seed(43052)
df = pd.DataFrame({'X1':['A']*5+['B']*5, 'X2':np.concatenate([np.random.randn(5), np.random.randn(5)+10])})
df

위의 자료에서 X1을 기준으로 그룹화한뒤 X2에 대하여 아래의 함수를
적용하라.

$$\max(\text{X2})-\min(\text{X2})$$

출력결과는 아래와 같아야 한다 .

In [166]:
# 